## Signal amplification by reversible exchange - pyridine: three spins

This is a simple example of a SABRE simulation of pyridine. The spin system consists of the hydride protons and one ligand proton. Chemical exchange and relaxation are not simulated. The simulation is performed near the energy level anticrossing field. Therefore, we find a coherent oscillation between the quantum states. This example takes a few seconds to run on a laptop with 11th generation i5 processor.

### Imports

We import NumPy, which is used for the arrays, and Matplotlib, which is used for plotting. The rest of the functionality comes from Spinguin.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from spinguin.spin_system import SpinSystem
from spinguin.hamiltonian import hamiltonian
from spinguin.propagation import propagator
from spinguin.states import singlet, measure

### Simulation parameters

Next, we assign the simulation parameters. We set the magnetic field to 5.3 mT, use 1 ms time step and set the number of time steps to 1000, for a total simulation time of 1 s.

In [ ]:
magnetic_field = 5.3e-3
time_step = 1e-3
nsteps = 1000

### Define the spin system

We must define the spin system, for which the simulations are going to be performed. We assign isotopes, chemical shifts (in ppm) and scalar couplings (in Hz) as NumPy arrays. Rest of the possible parameters, such as molecular coordinates, shielding tensors, etc. are not defined, as the simulation does not involve relaxation. Once the desired parameters are defined, they are passed to the spin system constructor.

In [ ]:
isotopes = np.array(['1H', '1H', '1H'])
chemical_shifts = np.array([-22.7, -22.7, 8.34])
scalar_couplings = np.array([\
    [ 0,     0,      0],
    [-6.53,  0,      0],
    [ 0.00,  1.66,   0]
])
spin_system = SpinSystem(isotopes, chemical_shifts, scalar_couplings)

## Calculate the Hamiltonian and the time propagator

Now the Hamiltonian can be calculated for the spin system. In addition, the magnetic field is passed as a parameter for the Hamiltonian function. This Hamiltonian, combined with the time step, is then used for calculating the time propagator.

In [ ]:
H = hamiltonian(spin_system, magnetic_field)
P = propagator(time_step, H)

### Assign the initial state

Initial state in this simulation is a singlet state for the hydride spins and thermal equilibrium for the substrate spin. It is assumed that the magnetization in thermal equilibrium is negligible.

In [ ]:
rho = singlet(spin_system, 0, 1)

### Create an empty array for storing the magnetizations

In the simulation, we are going to be performing 1000 time steps after each we will calculate the magnetization of each spin. Hence, we must create an empty array for storing the magnetizations during the time evolution.

In [ ]:
magnetizations = np.empty((nsteps, spin_system.size), dtype=complex)

### Perform the time evolution

Next, we will loop over the number of steps, and, during each step, we propagate the system forward in time and calculate the magnetization for each spin, which are saved to the previously created array.

In [ ]:
for step in range(nsteps):
    rho = P @ rho
    for i in range(isotopes.size):
        magnetizations[step, i] = measure(spin_system, rho, 'I_z', i)

### Plot the magnetizations and show the result

Finally, we will calculate the time axis, and plot the magnetizations of each spin as a function of time. We observe that the substrate spin obtains near full magnetization after approximately 0.5 s.

In [ ]:
t = np.linspace(time_step, nsteps*time_step, nsteps)
for i in range(isotopes.size):
    plt.plot(t, np.real(magnetizations[:,i]), label=f"Spin {i+1}")
plt.legend(loc="upper right")
plt.xlabel("Time (s)")
plt.ylabel("Magnetization")
plt.title("SABRE-hyperpolarization of Pyridine")
plt.tight_layout()
plt.show()
plt.clf()